In [ ]:
import requests
import json
import time


url = 'https://api.yelp.com/v3/businesses/search'


headers = {
    "accept": "application/json",
    "Authorization": f"Bearer {yelp_api_key}"
}

final_raw_data=[]

for i in range(0, 1000, 50):
    params = {
        "location": "Orange County",
        "term": "asian food",
        "sort_by": "best_match",
        "limit": 50,
        "offset": i
    }

    response = requests.get(url, headers=headers, params=params)
    raw_data = response.json()
    final_raw_data.extend(raw_data.get("businesses",[]))
    time.sleep(0.25)

final_raw_data


In [4]:
print ("Hello")

Hello
